# **Task P8**
## Analyzing Twitter Data using NLP

##### This task focuses on analyzing user replies to Covid-19 news published on the social media platform X (formerly known as Twitter) using some basic natural language processing (NLP) techniques. 

##### The task uses the `replies.csv` dataset, which contains the information about user replies to the Covid-19 news posted on Twitter. The columns capture the content related to the individual replies such as `text` (the actual text content of the reply), `id` (a unique ID for the reply itself), `conversation_id` (linking the reply to a specific conversation thread), `in_reply_to_user_id` (identifying the target user of the reply), and `possibly_sensitive` (indicating if the content may be sensitive). Additional informaiton include `author_id` (the unique ID of the reply's author), `author_followers` (the count of the author's followers),`author_tweets` (total tweets made by the author), and `author_location` (the geographical location of the author). Moreover, engagement metrics are tracked through columns like `retweets`, `replies`, `likes`, and `quote_count`. Time and location metadata are captured by `created_at` and `geo`, respectively, with `account` specifying the account name of the news publishers (e.g., BBC).

<img src="pic.png" alt="SIT112 News Tweets!" width="1800" height="200">

### A. Instructions

1. `To complete this task, you must first study the solution notebook; then try to do it yourself. You can copy the solutions into ChatGPT and ask for clarification or more examples.`
2. Complete and submit the TaskCompletionReport form in PDF format using Ontrack. You can do this by the end of Week 12, but feedback will only be provided for submissions received by 11 AM of the due date.
3. Do **not** include the solution (yours or the one provided to you) in your submission.
4. The workshops are the primary venue for you to seek help if you need any clarification/assitance. The tutors are there to help you complete and submit the tasks. Please avoid emailing your code or screenshots to the tutors outside workshop hours. 
5. Consider using online resources such as ChatGPT to strenghten your undertanding of the task. 

Good luck :)

In [1]:
import importlib
import subprocess

# The following lines should install all libraries you need - you can install the libraries manually if the script did not work 
required_modules = ['pandas', 'seaborn', 'matplotlib', 'numpy', 'datetime', 'sklearn', 'nltk', 'textblob', 'textblob', 'collections', 'numpy', 'warnings']
for module in required_modules:
    try:
        importlib.import_module(module)
    except ImportError:
        print(f"{module} module not found. Installing...")
        subprocess.check_call(['pip', 'install', module])

print("All required modules are installed.")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
import datetime as dt
from collections import Counter
import re
import warnings
import nltk
from nltk.corpus import stopwords

warnings.filterwarnings('ignore')
# Set the maximum number of columns and rows displayed
pd.set_option('display.max_columns', None)  # None means no limit
pd.set_option('display.max_rows', None)  # None means no limit
# You can also adjust the width of each column and the maximum column width
pd.set_option('display.max_colwidth', None)  # None means no truncation on column width

textblob module not found. Installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 7.5 MB/s eta 0:00:00
All required modules are installed.


### B. Write Python code that performs (answers) the following operations (questions) 

##### 1. Read the data from the .csv file into a DataFrame and display the first five rows.

In [3]:
# write your code here
data = pd.read_csv("replies.csv")
data.head(5)

,author_id,author_followers,author_tweets,author_location,id,conversation_id,source,in_reply_to_user_id,possibly_sensitive,text,created_at,geo,retweets,replies,likes,quote_count,account
0,2,1,3827,NaN,1.480000e+18,1.480000e+18,NaN,2.604917e+07,False,@RobinLaRoca @guardian I don't want a payment.\nI want the unvaxxed to pay every last red cent for public hospital treatment rendered to them,2022-01-01T03:31:03Z,NaN,0,2,0,0,guardian
1,3,939,49244,Sheffield,1.480000e+18,1.480000e+18,NaN,8.781841e+07,False,@guardian I'm fine with that as long as they backdate the payments for everyone.,2022-01-01T00:42:39Z,{'place_id': '07e9c7d1954fff64'},0,1,0,0,guardian
2,4,318,4519,"Santiago, Chile",1.480000e+18,1.480000e+18,NaN,8.781841e+07,False,"@guardian Yes, for big pharma",2022-01-01T00:37:59Z,NaN,1,0,1,0,guardian
3,5,2812,320641,Earth,1.480000e+18,1.480000e+18,NaN,1.367531e+06,False,@FoxNews a vaccine most Missourians won’t take,2022-01-01T07:39:51Z,NaN,0,0,5,0,FoxNews
4,6,350,1581,"Conway, AR",1.480000e+18,1.480000e+18,NaN,1.190000e+18,False,"@nikki6ixx1 @FoxNews I currently have Covid, im non vaccinated and feeling perfectly fine. Please explain to me how am I going to die because of this virus??",2022-01-01T13:30:39Z,NaN,0,2,0,0,FoxNews


##### 2. Clean the reply texts as listed below. You can use ChatGPT to generate regular expressions for these cleaning operations. 

- Remove @ Mentions: Strip out any @ mentions in the reply texts, which typically refer to other users and are not relevant for general analysis.
- Remove Hashtags: Eliminate the '#' symbol but keep the subsequent text, which might be relevant for understanding the topic of the reply.
- Remove Retweet Indicators: Remove any 'RT' symbols used to indicate retweets, as they do not contribute to text content.
- Remove URLs: Strip out any web links (URLs), which are common in reply texts but irrelevant to text analysis.
- Remove Punctuation: Eliminate all punctuation marks to reduce variability in the reply texts.
- Convert to Lowercase: Standardize all reply texts to lowercase to ensure uniformity in the analysis.

In [ ]:
# Write your code here
def remove_text(text):
    #removing @mentions from the text column
    text = re.sub(r"@\w+", "", text)

    #removing #s from the text
    text = re.sub(r"#", "", text)

    #remove retweet indicator "RT"
    text = re.sub(r"RT", "", text)
    
    #remove URLS
    text = re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "", text)
    
    #remove puntuation
    text = re.sub(r'[^\w\s]', "", text)

    #convert all the text to lower case
    text = text.lower()


data["text"] = data["text"].apply(remove_text)



##### 3. Remove from the reply texts the stopwords and the custom list of words that are unwanted:

- Utilize a list of predefined stopwords from NLTK tailored for the English language (from nltk.corpus import stopwords).
- Define a custom list of words to be excluded that are specifically irrelevant or overused in the reply texts.
- Remove stopwords and custom words: filter out these words from the (reply) text to focus on more meaningful words for analysis.

In [4]:
# Write your code here



##### 4. What are the 5 most frequent words in the replies per account (news publisher, e.g., BBC)?

In [5]:
# Write your code here



##### 5. Classify the sentiment of each reply as positive, negative, or neutral based on the `text` column. Then visualize the ratio of the replies with positive, negative, or neutral sentiments per news publisher (account). 


In [6]:
# Write your code here



### C. Learn more
* https://www.sciencedirect.com/science/article/pii/S2468696423000125
* https://youtu.be/toM78JGJ_4s?si=8ESJ-yOV_H1iX74r
* https://youtu.be/QpzMWQvxXWk?si=CPvNgnZo3jsW4HL5
* https://youtu.be/o7OqhzMcDfs?si=9iFeYZq4TlVCMMRb